# Kaggle Top Spotify songs from 2010-2019

https://www.kaggle.com/leonardopena/top-spotify-songs-from-20102019-by-year

## Objectiu

Utilitzar la base de dades de Kaggle del Top Spotify songs from 2010-2019 per entrenar un classificador que ens digui a quin percentatge de popularitat pot pèrtanyer una nova canço de spotify segons les seves dades. És a dir, pel nostre case el target serà l'atribut "pop".

## Continguts

1. EDA i Preprocessing
* Anàlisi de tamany i tipologia de dades
* Encoding de categòriques
* Valoració de correlacions
* Normalització

2. Metric Selection
* Separació de les dades
* Selecció de la millor mètrica
* Visualització de ROC/AUC

3. Model Selection i Crossvalidation
* Selecció del millor model
* Cerca d'hiperparàmetres

4. Anàlisi Final

Primer de tot, importem llibreries essencials, a més de les dades i observem que s'han importat correctament. Ho veiem a continuació:

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder

df=pd.read_csv("top10s.csv", encoding='latin1')

print(df.dtypes)
print(df.head())

Unnamed: 0     int64
title         object
artist        object
top genre     object
year           int64
bpm            int64
nrgy           int64
dnce           int64
dB             int64
live           int64
val            int64
dur            int64
acous          int64
spch           int64
pop            int64
dtype: object
   Unnamed: 0                 title      artist        top genre  year  bpm  \
0           1      Hey, Soul Sister       Train       neo mellow  2010   97   
1           2  Love The Way You Lie      Eminem  detroit hip hop  2010   87   
2           3               TiK ToK       Kesha        dance pop  2010  120   
3           4           Bad Romance   Lady Gaga        dance pop  2010  119   
4           5  Just the Way You Are  Bruno Mars              pop  2010  109   

   nrgy  dnce  dB  live  val  dur  acous  spch  pop  
0    89    67  -4     8   80  217     19     4   83  
1    93    75  -5    52   64  263     24    23   82  
2    84    76  -3    29   71  200 

## 1. EDA i Preprocessing

### - Anàlisi de tamany i tipologia de dades

Mirem quantes mostres i quants atributs té la nostre base de dades i què significa cadascun.

In [20]:
print("Número de mostres:", df.shape[0])
print("Número d'atributs:", df.columns.shape[0])

Número de mostres: 603
Número d'atributs: 15


-Unnamed: ID

-title: Song's title

-artist: Song's artist

-top genre: The genre of the track

-year: Song's year in the Billboard

-bpm: Beats.Per.Minute - The tempo of the song

-nrgy: Energy- The energy of a song - the higher the value, the more energtic. song

-dnce: Danceability - The higher the value, the easier it is to dance to this song

-dB: Loudness..dB.. - The higher the value, the louder the song

-live: Liveness - The higher the value, the more likely the song is a live recording

-val: Valence - The higher the value, the more positive mood for the song

-dur: Length - The duration of the song

-acous: Acousticness.. - The higher the value the more acoustic the song is

-spch: Speechiness - The higher the value the more spoken word the song contains

-pop: Popularity- The higher the value the more popular the song is

El tipus d'atributs que tenim són:

In [6]:
print(df.dtypes)

Unnamed: 0     int64
title         object
artist        object
top genre     object
year           int64
bpm            int64
nrgy           int64
dnce           int64
dB             int64
live           int64
val            int64
dur            int64
acous          int64
spch           int64
pop            int64
dtype: object


-Categòrics: title, artist i top genre
    
-Numèrics: ID, year, bpm, nrgy, dnce, dB, live, val, dur, acous, spch i pop

Tenim NaNs a les dades?

In [8]:
print(df.isna().sum())

Unnamed: 0    0
title         0
artist        0
top genre     0
year          0
bpm           0
nrgy          0
dnce          0
dB            0
live          0
val           0
dur           0
acous         0
spch          0
pop           0
dtype: int64


Sorprenentment no tenim NaNs a les dades, així que no hem de tractar res en aquest cas.

### - Encoding de categòriques

Anem a analitzar una mica més les variables categòriques. Analitzem quines cançons es repeteixen perquè apareixen en anys diferents i mirem si l'artista i popularitat coincideixen.

In [44]:
# Contem quantes vegades es repeteix cada títol
frequencia_titol = df['title'].value_counts()

# Filtrem els títols que es repeteixen més d'una vegada
titols_repetits = frequencia_titol[frequencia_titol > 1].index

# Filtrem les files que corresponen a aquests titols repetits
files_repetides = df[df['title'].isin(titols_repetits)]

# Mostrem les files
print(files_repetides[['title', 'artist', 'pop']].sort_values(by='title'))

                                               title             artist  pop
239  A Little Party Never Killed Nobody (All We Got)             Fergie   65
178  A Little Party Never Killed Nobody (All We Got)             Fergie   65
398                                        All I Ask              Adele   71
473                                        All I Ask              Adele   71
95           Castle Walls (feat. Christina Aguilera)               T.I.   49
48           Castle Walls (feat. Christina Aguilera)               T.I.   49
301                                          Company      Justin Bieber   72
396                                          Company      Justin Bieber   72
548                                       First Time               Kygo   71
474                                       First Time               Kygo   71
295                                            Hello              Adele   73
50                                             Hello     Martin Solveig    0

In [45]:
print(df['artist'].value_counts())

artist
Katy Perry             17
Justin Bieber          16
Rihanna                15
Maroon 5               15
Lady Gaga              14
                       ..
Iggy Azalea             1
5 Seconds of Summer     1
Michael Jackson         1
Disclosure              1
Daddy Yankee            1
Name: count, Length: 184, dtype: int64
